In [2]:
!pip install -U bitsandbytes


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import transformers
import inseq
import torch
import pandas as pd

/home/ada/humor/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
transcripts = pd.read_csv('/home/ada/humor/data/stand_up_dataset/standup_transcripts.csv')

In [5]:
transcripts.head()

,comedian,transcript
0,Anthony_Jeselnik,"When I was a kid, I used to fantasize about ge..."
1,Anthony_Jeselnik_2,No one should ever ask me to speak at anyone’...
2,Ali_Wong,"The last time I was at home in San Francisco, ..."
3,Ali_Wong_2,I need to have children to keep me company whe...
4,Chelsea_Peretti,A friend of a friend just posted like 500 enga...


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ada/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import nltk.data

def split_transcript(transcript):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = tokenizer.tokenize(transcript.strip())
    total_sentences = len(sentences)
    sentences_per_part = total_sentences // 4

    part1 = sentences[:sentences_per_part]
    part2 = sentences[sentences_per_part:2*sentences_per_part]
    part3 = sentences[2*sentences_per_part:3*sentences_per_part]
    part4 = sentences[3*sentences_per_part:]

    part1_text = ' '.join(part1)
    part2_text = ' '.join(part2)
    part3_text = ' '.join(part3)
    part4_text = ' '.join(part4)

    return [part1_text, part2_text, part3_text, part4_text]


In [8]:
transcript_ali_wong = transcripts.loc[transcripts['comedian'] == 'Ali_Wong', 'transcript'].values[0]
parts = split_transcript(transcript_ali_wong)

In [9]:
Q8BIT = transformers.BitsAndBytesConfig(load_in_8bit=True)

model = inseq.load_model(
    model="google/gemma-2b-it", 
    attribution_method="input_x_gradient",
    model_kwargs=dict(
        device_map="cuda:0",
        torch_dtype=torch.float32,
        trust_remote_code=False,
        low_cpu_mem_usage=True,
        attn_implementation="eager",
        quantization_config=Q8BIT,
    )
)

print("memory cost: ", model.model.get_memory_footprint() >> 20, "Mb")

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.10s/it]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.


memory cost:  3890 Mb


In [11]:
prompt = f"This is a stand-up comedy transcript: '''{parts[1]}'''\nExtract the key humorous lines and punchlines:"

out = model.attribute(
prompt,
n_steps=16,
batch_size=1,
generation_args=dict(
    max_new_tokens=65,
)
)
()
out.show()

Unused arguments during attribution: {'n_steps': 16}


OutOfMemoryError: CUDA out of memory. Tried to allocate 104.00 MiB (GPU 0; 23.58 GiB total capacity; 6.93 GiB already allocated; 15.19 MiB free; 7.38 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
out.sequence_attributions[0].target_attributions.sum(-1)

array([[ 9.3318298e-03, -2.5626723e-02, -2.4753304e-02, ...,
        -5.7015467e-02,  4.4034969e-06, -1.4212786e-04],
       [-8.6237743e-02,  5.4679070e-02, -2.1457385e-02, ...,
         6.9233894e-02, -2.9941932e-05, -2.6500195e-05],
       [ 1.3873137e-03,  2.1565673e-03, -4.9037402e-03, ...,
        -2.7292579e-02, -4.5716383e-06,  1.6739892e-05],
       ...,
       [           nan,            nan,            nan, ...,
                   nan, -1.1118839e-04,  2.4466758e-04],
       [           nan,            nan,            nan, ...,
                   nan,            nan, -3.3755862e-04],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan]], dtype=float32)

In [ ]:
# for part in parts:
#     prompt = f"This is a stand-up comedy transcript: '''{part}'''\nExtract the key humorous lines and punchlines:"

#     out = model.attribute(
#     prompt,
#     n_steps=16,
#     batch_size=1,
#     device=model.device,
#     generation_args=dict(
#         max_new_tokens=65,
#     )
#     )

#     out.show()

In [ ]:
results = {}
for i in range(len(transcripts['comedian'])): 
    comedian = transcripts['comedian'][i]
    trans = transcripts['transcript'][i]
    
    parts = split_transcript(trans)
    results[comedian] = []
    
    for part in parts:
        prompt = f"This is a stand-up comedy transcript: '''{part}'''\nExtract the key humorous lines and punchlines:"

        out = model.attribute(
        prompt,
        n_steps=16,
        batch_size=1,
        device=model.device,
        generation_args=dict(
            max_new_tokens=65,
        )
        )
        results[comedian].append(out)
    break

In [ ]:
results["Anthony_Jeselnik"][0].sequence_attributions[0].target_attributions.sum(-1)